In [1]:
from __future__ import print_function, division

#### 자료 안내: 여기서 다루는 내용은 아래 사이트의 내용을 참고하여 생성되었음.

https://github.com/rouseguy/intro2stats

# 표본분포와 신뢰구간

## 주요내용

앞서 [GongSu22](https://github.com/liganega/Gongsu-DataSci/blob/master/Notes/W10/GongSu22_Statistics_Population_Variance.ipynb)에서 표본 데이터를 이용하여 모집단의 평균과 분산에 대한 점추정을 알아보았다.
여기서는 표본분포를 이용하여 미국 51개 주에서 거래된 담배(식물) 도매가의 평균에 대한 신뢰구간을 구하는 방법을 알아본다.

## 주요 예제

* 캘리포니아 주에서 2014년에 거래된 상품 담배(식물)의 도매가
* 도매가의 표본분포 구하기
* 도매가의 평균값 추정치의 신뢰구간 구하기

## 주요 모듈

numpy와 pandas 이외에 통계전용 모듈인 stats 모듈을 임포트 한다.

In [2]:
import numpy as np
import pandas as pd
from scipy import stats

## 연도별, 월별 데이터 추출하기

Weed_Price.csv의 date 열에는 거래일자가 2014-01-01의 형식으로 포함되어 있다.
이 정보를 이용하여 연도별, 월별 데이터를 구하려면 거래일자로부터 연도 또는 월에 대한 정보만을 추출할 수 있어야 한다.

여기서는 Timestamp 자료형의 속성을 활용하여 필요한 정보를 얻는 방식을 배운다.

### csv 파일 불러오기
 
먼저 Weed_Price.csv 파일의 내용을 다시 불러 온다.

In [4]:
weed_pd = pd.read_csv("data/Weed_Price.csv", parse_dates=[-1])
weed_pd.head()

,State,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN,date
0,Alabama,339.06,1042,198.64,933,149.49,123,2014-01-01
1,Alaska,288.75,252,260.60,297,388.58,26,2014-01-01
2,Arizona,303.31,1941,209.35,1625,189.45,222,2014-01-01
3,Arkansas,361.85,576,185.62,544,125.87,112,2014-01-01
4,California,248.78,12096,193.56,12812,192.92,778,2014-01-01


In [5]:
weed_pd.tail()

,State,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN,date
22894,Virginia,364.98,3513,293.12,3079,NaN,284,2014-12-31
22895,Washington,233.05,3337,189.92,3562,NaN,160,2014-12-31
22896,West Virginia,359.35,551,224.03,545,NaN,60,2014-12-31
22897,Wisconsin,350.52,2244,272.71,2221,NaN,167,2014-12-31
22898,Wyoming,322.27,131,351.86,197,NaN,12,2014-12-31


In [12]:
weed_pd.dtypes

State             object
HighQ            float64
HighQN             int64
MedQ             float64
MedQN              int64
LowQ             float64
LowQN              int64
date      datetime64[ns]
dtype: object

### Timestamp 자료형

date 열에는 거래일자 정보가 들어 있는데, 각 정보의 자료형은 Timestamp 라고 불린다.
 
0번 줄의 date 정보를 확인하면 Timestamp 자료형이 사용된 것을 확인할 수 있다.

In [16]:
weed_pd.date.head()

0   2014-01-01
1   2014-01-01
2   2014-01-01
3   2014-01-01
4   2014-01-01
Name: date, dtype: datetime64[ns]

In [18]:
weed_pd.date[0]

Timestamp('2014-01-01 00:00:00')

Timestamp 자료형에는 year, month, day 등 거래일자에 대한 구체적인 구체적인 정보를 담고 있는 속성이 포함되어 있다.

In [5]:
weed_pd.date[0].year

2014

In [6]:
weed_pd.date[0].month

1

In [7]:
weed_pd.date[0].day

1

### apply 함수 활용

위 속성들을 이용하여 거래일자에서 연도별, 월별 정보를 추출하여 새로운 열(컬럼)으로 추가할 수 있다.
이를 위해 apply() 함수를 활용한다.
 
**주의:** Series와 DataFrame 자료형에 apply 함수와 동일한 기능을 가진 apply 메소드가 선언되어 있다.

먼저 weed_pd.date 의 자료형이 Series 임을 확인한다.

In [8]:
type(weed_pd.date)

pandas.core.series.Series

`x`가 Timestamp 자료형일 때, `x`에서 연도 정보를 추출하는 함수를 아래와 같이 정의할 수 있다.

In [20]:
def getYear(x):
    return x.year

동일한 방식으로 x가 Timestamp 자료형일 때, x 에서 월에 정보를 추출하는 함수를 아래와 같이 정의할 수 있다.

In [21]:
def getMonth(x):
    return x.month

이제 두 함수를 이용하여 거래년도 만을 담는 칸을 추가할 수 있다. 
 
먼저 아래 코드를 실행해보자.

In [23]:
year_col = weed_pd.date.apply(getYear)
year_col.head()

0    2014
1    2014
2    2014
3    2014
4    2014
Name: date, dtype: int64

위 결과의 자료형은 Series 이다.

In [24]:
type(year_col)

pandas.core.series.Series

동일한 방식으로 month_col을 추출한다.

In [25]:
month_col = weed_pd.date.apply(getMonth)
month_col.head()

0    1
1    1
2    1
3    1
4    1
Name: date, dtype: int64

### DataFrame에 열 추가하기
 
추출한 두 시리즈를 weed_pd 에 새로운 열로 추가한다.

In [27]:
weed_pd["month"] = month_col
weed_pd["year"] = year_col

두 개의 열이 추가되었음을 확인할 수 있다.

In [28]:
weed_pd.head()

,State,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN,date,month,year
0,Alabama,339.06,1042,198.64,933,149.49,123,2014-01-01,1,2014
1,Alaska,288.75,252,260.60,297,388.58,26,2014-01-01,1,2014
2,Arizona,303.31,1941,209.35,1625,189.45,222,2014-01-01,1,2014
3,Arkansas,361.85,576,185.62,544,125.87,112,2014-01-01,1,2014
4,California,248.78,12096,193.56,12812,192.92,778,2014-01-01,1,2014


## 캘리포니아 주에서 2014년도 거래된 담배(식물) 도매가 추출하기

마스크 인덱싱을 활용하여 캘리포니아 주에서 2014년도에 거래된 데이터만 추출할 수 있다.

**주의:** weed_pd에 year 열을 추가하였기에 아래 방식이 가능하다.

In [30]:
weed_ca_2014 = weed_pd[(weed_pd.State=="California") & (weed_pd.year==2014)]
weed_ca_2014.head()

,State,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN,date,month,year
4,California,248.78,12096,193.56,12812,192.92,778,2014-01-01,1,2014
106,California,248.20,12571,192.80,13406,191.94,804,2014-02-01,2,2014
208,California,247.60,12988,192.97,13906,191.40,839,2014-03-01,3,2014
259,California,246.76,13396,192.91,14527,191.98,863,2014-04-01,4,2014
310,California,246.04,13787,191.89,15047,191.40,891,2014-05-01,5,2014


### 캘리포니아 주에서 2014년도에 거래된 상품 담배(식물)의 도매가의 평균분포

#### 평균분포의 평균값

In [32]:
ca_2014_mean = weed_ca_2014.HighQ.mean()
ca_2014_mean

245.8942307692309

#### 평균분포의 표준편차

In [33]:
ca_2014_std = weed_ca_2014.HighQ.std()
ca_2014_std

1.289907939371412

#### 신뢰구간

신뢰수준 95%에 대한 신뢰구간을 구할 수 있다.

In [34]:
stats.norm.interval(0.95, loc=ca_2014_mean, scale = ca_2014_std/np.sqrt(len(weed_ca_2014)))

(245.761718492726, 246.02674304573577)

신뢰구간 설명: Weed_Price.csv 파일에는 거래된 담배의 도매가의 일부 데이터들의 표본을 담고 있다. 
하지만 이 정보를 이용하여 미국 전체에서 거래된 모든 모대가에 대한 정보를 추정할 수 있다. 
이를 위해 표본분포를 활용하며, 앞서 구한 신뢰구간의 의미는 다음과 같다.

> 캘리포니아 주에서 2014년도에 거래된 모든 상품(HighQ) 담배(식물)의 도매가의 평균값은 앞서 구한 신뢰구간에 위치할 확률이 95%이다.